import needed libraries

In [203]:
import random
import math
import numpy as np

This class is for test case objects that saves different values of each object

In [232]:
class Data:

    # initialize needed values
    def __init__(self, data_address):
        self.address    = data_address      # address of test file
        self.degree     = 0                 # degree of polynomial
        self.low        = 0                 # lower bound of coefs
        self.high       = 0                 # upper bound of coefs
        self.num_points = 0                 # number of points in test case
        self.points     = []                # points of curve

    # read test case
    def load_data(self):
        with open (self.address, 'r') as f:
            self.degree         = int(f.readline())     
            self.low, self.high = map(float, f.readline().split(" "))  
            self.num_points     = int(f.readline())  
            self.points         = [tuple(map(float, f.readline().split(" "))) for _ in range(self.num_points)]  # append the point to list of points  


a class for gene objects, with different methods and values of a gene

In [345]:
class Gene:

    # initialize parameters
    def __init__(self, degree, points, low, high):
        self.gene    = []
        self.degree  = degree
        self.points  = points
        self.low     = low
        self.high    = high
        self.fitness = math.inf


    # create random gene for first population
    def initial_gene(self):
        self.gene = [random.uniform(self.low, self.high) for _ in range(self.degree)]

    # calculate fitness of the gene
    def fitness_function(self):
        y_pred       = [np.polyval(self.gene, point[0]) for point in self.points]  # predicted value for each point
        residuals    = [(y_pred[i] - self.points[i][1])**2 for i in range(len(self.points))]  # SSE for each point
        weight       = 1.0 / np.std(y_pred)  
        weighted_rss = weight * np.sum(residuals)   # calculate weighted RSS

        self.fitness = np.sqrt(weighted_rss) / 150
        return self.fitness
    
    # mutate gene
    def mutation(self):

        i = random.randint(0, (self.degree-1)//2)
        j = random.randint((self.degree-1)//2, self.degree-1)
        k = random.randint((self.degree-1)//2, self.degree-1)

        # randomly change 3 coefs in gene
        self.gene[i] = np.round(random.uniform(-10, 10), 6)
        self.gene[j] = np.round(random.uniform(-10, 10), 6)
        self.gene[k] = np.round(random.uniform(-10, 10), 6)
        
        self.fitness = self.fitness_function()


    # define a local search to search for neighbors of a gene that is better than itself
    def local_search(self):

        neighbors = []
        for i in range(self.degree):   # find as many neighbors as degree of polynimal and change one index at a time
            genei = Gene(self.degree, self.points, self.low, self.high)
            rnd = np.round(random.uniform(-10, 10), 6)
            genei.gene = self.gene
            genei.gene[i] = rnd
            genei.fitness = genei.fitness_function()
            neighbors.append((genei.fitness, genei))

        # sort neighbors and select the best one or the child itself
        neighbors = sorted(neighbors, key=lambda x: x[0])
        if neighbors[0][0] < self.fitness:
            return neighbors[0][1]
        else:
            return self



a class for population of genes

In [298]:
class Population:

    # initialize population's parameters
    def __init__(self, data, pop_size=100):
        self.population_size             = pop_size
        self.population                  = []
        self.points                  = data.points
        self.low, self.high, self.degree = data.low, data.high, data.degree

    # generate first population randomly
    def initialize_population(self):
        for _ in range(self.population_size):
            genei = Gene(self.degree, self.points, self.low, self.high)
            genei.initial_gene()
            self.population.append(genei)
    

    # selest parents from population for crossover using tournament selection
    def parent_selection(self, N=3):
        # choose random N candidates to attend tournament
        father_candidates = [random.choice(self.population) for _ in range(N)]
        mother_candidates = [random.choice(self.population) for _ in range(N)]

        # choose the best one of N candidates to be a parent
        father_vals = [(father.fitness, father) for father in father_candidates]
        mother_vals = [(mother.fitness, mother) for mother in mother_candidates]

        # sort candidates
        sorted_fathers = sorted(father_vals, key=lambda x: x[0])
        sorted_mothers = sorted(mother_vals, key=lambda x: x[0])

        # select parents with min fitness
        best_father = sorted_fathers[0][1]
        best_mother = sorted_mothers[0][1]
        
        return best_father, best_mother

    # selest parents from population for crossover using rank-based selection
    def parent_selection_rank_based(self):
        # sort population based on fitness
        fitness_values = [gene.fitness for gene in self.population]
        ranked_indices = np.argsort(fitness_values)
        ranked_population = [self.population[i] for i in ranked_indices]
        
        # rank calculation for selection 
        selection_p = np.arange(1, self.population_size + 1) / np.sum(np.arange(1, self.population_size + 1))
        
        # select parents based on rank and selection probabilities
        selected_parents = np.random.choice(ranked_population, size=2, replace=False, p=selection_p)
        
        return selected_parents[0], selected_parents[1]


    # crossover function (order)
    def crossover(self, father, mother):

        child1l, child2l = [], []
        child1, child2   = Gene(self.degree, self.points, self.low, self.high), Gene(self.degree, self.points, self.low, self.high)

        random_point = random.randint(0, (self.degree-1)//2)    # select a random point and cut 1 of parents for each child from there

        # put first part of 1 parent to each child
        child1l = [father.gene[i] for i in range(random_point)]
        child2l = [mother.gene[i] for i in range(random_point)]

        # put the rest of other parent
        for i in range(random_point, self.degree):
            child1l.append(mother.gene[i])
            child2l.append(father.gene[i])

        # initialize childs gene object
        child1.gene    = child1l
        child2.gene    = child2l
        child1.fitness = child1.fitness_function()
        child2.fitness = child2.fitness_function()
        
        # select the best generated child
        if child1.fitness < child2.fitness:
            return child1
        else:
            return child2
    
    # blend crossover
    def blx_alpha_crossover(self, father, mother, alpha=0.5):
        child = []
        for i in range(self.degree):
            min_val = min(father.gene[i], mother.gene[i])
            max_val = max(father.gene[i], mother.gene[i])
            
            parameter_range = max_val - min_val
            lower_bound     = min_val - alpha * parameter_range
            upper_bound     = max_val + alpha * parameter_range
            
            # check domain
            if lower_bound < self.low:
                lower_bound = self.low
            if upper_bound > self.high:
                upper_bound = self.high

            # generate a random value within the range 
            child_val = np.random.uniform(lower_bound, upper_bound)
            child.append(child_val)
        
        # initialize child object
        child1 = Gene(self.degree, self.points, self.low, self.high)
        child1.gene = child
        child1.fitness = child1.fitness_function()
        return child1

    # average crossover function
    def avg_crossover(self, father, mother):

        child1l = []
        child1 = Gene(self.degree, self.points, self.low, self.high)

        # put first part of 1 parent to each child
        child1l = []

        # put the rest of other parent
        for i in range(self.degree):
            child1l.append((father.gene[i] + mother.gene[i]) / 2)

        # initialize childs gene object
        child1.gene = child1l
        child1.fitness = child1.fitness_function()
        
        return child1


    # random bit crossover function
    def rnd_crossover(self, father, mother):

        child1l = []
        child1 = Gene(self.degree, self.points, self.low, self.high)

        # specify the prob of selecting each parent's gene based on their fitness
        if father.fitness > mother.fitness:
            p_select = 0.3
        else:
            p_select = 0.7

        # put mother or father's gene in child gene with prob p_select
        for i in range(self.degree):
            e_select = random.random()
            if e_select > p_select:
                child1l[i] = father.gene[i]
            else:
                child1l[i] = mother.gene[i]


        # initialize childs gene object
        child1.gene = child1l
        child1.fitness = child1.fitness_function()
        
        return child1



### Genetic (curve fitting)

In [316]:
def genetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_replace, pop_size, stop_iter, expected_val, tornoment_N=3):
    # save best seen result
    best_cost = math.inf
    best_result = []

    # load test case
    testcase = Data(test_address)
    testcase.load_data()

    # step 1: create first population
    population = Population(testcase, pop_size=pop_size)
    population.initialize_population()

    iteration_without_change, g_num = 0, 0
    # iteration of running algorithm (number of generations)
    while True:
        g_num += 1 # iteration number
        children_population = Population(testcase, pop_size=pop_size)
        counter = 0
        # create as many children as parents population 
        while counter < pop_size:

            # step 2: parent selection for crossover
            father1, mother1 = population.parent_selection_rank_based()
            father2, mother2 = population.parent_selection_rank_based()

            # step 3: crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                child1 = population.blx_alpha_crossover(father1, mother1, alpha=0.6)
                child2 = population.blx_alpha_crossover(father2, mother2, alpha=0.65)
                counter += 2

                # step 4: mutate children with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                # add generated children to children population
                children_population.population.append(child1)
                children_population.population.append(child2)
        
        # step 5: population replacement
        parent_values = []
        child_values  = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness
            y = children_population.population[i].fitness
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        # replace num_replace worst parent with best children
        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))    # find the worst parent
            min_child_ind  = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # step 6: population fitness check, find the best result in population
        flag = False
        for i in range(pop_size):
            if population.population[i].fitness < best_cost:
                best_cost   = population.population[i].fitness
                best_result = population.population[i].gene.copy()
                iteration_without_change = 0
                flag = True
        if not flag:
            iteration_without_change += 1

        print(f"Iteration {g_num}, best_cost so far: {np.round(best_cost, 3)}")

        # step 7: check stop condition
        if best_cost < expected_val or iteration_without_change > stop_iter:
            break


    return best_result, testcase

            


Test 1

In [296]:
best_result, test = genetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test1.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=200, stop_iter=100, expected_val=100)

Iteration 1, best_cost so far: 57693.328
Iteration 2, best_cost so far: 19596.695
Iteration 3, best_cost so far: 19596.695
Iteration 4, best_cost so far: 19596.695
Iteration 5, best_cost so far: 9519.209
Iteration 6, best_cost so far: 9519.209
Iteration 7, best_cost so far: 9519.209
Iteration 8, best_cost so far: 9519.209
Iteration 9, best_cost so far: 9519.209
Iteration 10, best_cost so far: 9519.209
Iteration 11, best_cost so far: 9519.209
Iteration 12, best_cost so far: 9519.209
Iteration 13, best_cost so far: 8307.142
Iteration 14, best_cost so far: 8307.142
Iteration 15, best_cost so far: 8307.142
Iteration 16, best_cost so far: 418.916
Iteration 17, best_cost so far: 418.916
Iteration 18, best_cost so far: 418.916
Iteration 19, best_cost so far: 418.916
Iteration 20, best_cost so far: 418.916
Iteration 21, best_cost so far: 418.916
Iteration 22, best_cost so far: 418.916
Iteration 23, best_cost so far: 418.916
Iteration 24, best_cost so far: 418.916
Iteration 25, best_cost so far

Test 2

In [327]:
best_result, test = genetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test2.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=200, stop_iter=100, expected_val=100)

Iteration 1, best_cost so far: 14540.922
Iteration 2, best_cost so far: 11935.463
Iteration 3, best_cost so far: 11935.463
Iteration 4, best_cost so far: 11935.463
Iteration 5, best_cost so far: 11935.463
Iteration 6, best_cost so far: 1933.533
Iteration 7, best_cost so far: 1933.533
Iteration 8, best_cost so far: 1933.533
Iteration 9, best_cost so far: 1933.533
Iteration 10, best_cost so far: 1933.533
Iteration 11, best_cost so far: 1933.533
Iteration 12, best_cost so far: 1933.533
Iteration 13, best_cost so far: 1933.533
Iteration 14, best_cost so far: 1933.533
Iteration 15, best_cost so far: 1933.533
Iteration 16, best_cost so far: 1933.533
Iteration 17, best_cost so far: 1853.855
Iteration 18, best_cost so far: 1853.855
Iteration 19, best_cost so far: 1853.855
Iteration 20, best_cost so far: 1608.162
Iteration 21, best_cost so far: 1455.716
Iteration 22, best_cost so far: 1455.716
Iteration 23, best_cost so far: 1455.716
Iteration 24, best_cost so far: 1455.716
Iteration 25, best_c

Test 3

In [325]:
best_result, test = genetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test3.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, stop_iter=100, expected_val=100)

Iteration 1, best_cost so far: 64676.956
Iteration 2, best_cost so far: 21426.365
Iteration 3, best_cost so far: 21426.365
Iteration 4, best_cost so far: 21426.365
Iteration 5, best_cost so far: 21426.365
Iteration 6, best_cost so far: 21426.365
Iteration 7, best_cost so far: 21426.365
Iteration 8, best_cost so far: 21426.365
Iteration 9, best_cost so far: 10580.094
Iteration 10, best_cost so far: 10580.094
Iteration 11, best_cost so far: 10580.094
Iteration 12, best_cost so far: 10580.094
Iteration 13, best_cost so far: 10580.094
Iteration 14, best_cost so far: 10580.094
Iteration 15, best_cost so far: 10580.094
Iteration 16, best_cost so far: 10580.094
Iteration 17, best_cost so far: 10580.094
Iteration 18, best_cost so far: 10580.094
Iteration 19, best_cost so far: 10580.094
Iteration 20, best_cost so far: 10580.094
Iteration 21, best_cost so far: 10580.094
Iteration 22, best_cost so far: 10580.094
Iteration 23, best_cost so far: 10580.094
Iteration 24, best_cost so far: 10580.094
I

Test 4

In [331]:
best_result, test = genetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test4.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=200, stop_iter=100, expected_val=1)

Iteration 1, best_cost so far: 1364.578
Iteration 2, best_cost so far: 1364.578
Iteration 3, best_cost so far: 1364.578
Iteration 4, best_cost so far: 1364.578
Iteration 5, best_cost so far: 1364.578
Iteration 6, best_cost so far: 1364.578
Iteration 7, best_cost so far: 1364.578
Iteration 8, best_cost so far: 960.284
Iteration 9, best_cost so far: 290.336
Iteration 10, best_cost so far: 290.336
Iteration 11, best_cost so far: 290.336
Iteration 12, best_cost so far: 290.336
Iteration 13, best_cost so far: 290.336
Iteration 14, best_cost so far: 290.336
Iteration 15, best_cost so far: 290.336
Iteration 16, best_cost so far: 276.282
Iteration 17, best_cost so far: 201.766
Iteration 18, best_cost so far: 201.766
Iteration 19, best_cost so far: 201.766
Iteration 20, best_cost so far: 201.766
Iteration 21, best_cost so far: 201.766
Iteration 22, best_cost so far: 201.766
Iteration 23, best_cost so far: 201.766
Iteration 24, best_cost so far: 201.766
Iteration 25, best_cost so far: 137.942
It

### Memetic (curve fitting)

In [333]:
def memetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_neighbor, p_replace, pop_size, stop_iter, expected_val, tornoment_N=3):
    # save best seen result
    best_cost = math.inf
    best_result = []

    # load test case
    testcase = Data(test_address)
    testcase.load_data()

    # step 1: create first population
    population = Population(testcase, pop_size=pop_size)
    population.initialize_population()

    iteration_without_change, g_num = 0, 0
    # iteration of running algorithm (number of generations)
    while True:
        g_num += 1 # iteration number
        children_population = Population(testcase, pop_size=pop_size)
        counter = 0
        # create as many children as parents population 
        while counter < pop_size:

            # step 2: parent selection for crossover
            father1, mother1 = population.parent_selection_rank_based()
            father2, mother2 = population.parent_selection_rank_based()

            # step 3: crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                child1 = population.blx_alpha_crossover(father1, mother1, alpha=0.6)
                child2 = population.blx_alpha_crossover(father2, mother2, alpha=0.65)
                counter += 2

                # step 4: mutate children with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                # step 5: local search on children and update fitness
                e_neighbor1 = random.random()
                if e_neighbor1 < p_neighbor:
                    genei = child1.local_search()
                else:
                    genei = child1
                e_neighbor2 = random.random()
                if e_neighbor2 < p_neighbor:
                    genej = child2.local_search()
                else:
                    genej = child2
                

                # add generated children to children population
                children_population.population.append(genei)
                children_population.population.append(genej)
        
        # step 6: population replacement
        parent_values = []
        child_values  = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness
            y = children_population.population[i].fitness
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        # replace num_replace worst parent with best children
        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))    # find the worst parent
            min_child_ind  = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # step 7: population fitness check, find the best result in population
        flag = False
        for i in range(pop_size):
            if population.population[i].fitness < best_cost:
                best_cost   = population.population[i].fitness
                best_result = population.population[i].gene.copy()
                iteration_without_change = 0
                flag = True
        if not flag:
            iteration_without_change += 1

        print(f"Iteration {g_num}, best_cost so far: {np.round(best_cost, 3)}")

        # step 7: check stop condition
        if best_cost < expected_val or iteration_without_change > stop_iter:
            break


    return best_result, testcase

            


Test 1

In [342]:
best_result, test = memetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test1.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, stop_iter=100, expected_val=100, p_neighbor=0.5)

Iteration 1, best_cost so far: 28727.325
Iteration 2, best_cost so far: 28727.325
Iteration 3, best_cost so far: 28727.325
Iteration 4, best_cost so far: 23192.152
Iteration 5, best_cost so far: 23192.152
Iteration 6, best_cost so far: 23192.152
Iteration 7, best_cost so far: 23192.152
Iteration 8, best_cost so far: 23192.152
Iteration 9, best_cost so far: 23192.152
Iteration 10, best_cost so far: 10912.881
Iteration 11, best_cost so far: 10912.881
Iteration 12, best_cost so far: 10912.881
Iteration 13, best_cost so far: 4995.811
Iteration 14, best_cost so far: 4508.999
Iteration 15, best_cost so far: 4508.999
Iteration 16, best_cost so far: 4508.999
Iteration 17, best_cost so far: 2896.726
Iteration 18, best_cost so far: 2896.726
Iteration 19, best_cost so far: 2896.726
Iteration 20, best_cost so far: 2896.726
Iteration 21, best_cost so far: 2896.726
Iteration 22, best_cost so far: 2896.726
Iteration 23, best_cost so far: 2896.726
Iteration 24, best_cost so far: 2896.726
Iteration 25,

Test 2

In [346]:
best_result, test = memetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test2.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, stop_iter=100, expected_val=100, p_neighbor=0.5)

Iteration 1, best_cost so far: 13188.961
Iteration 2, best_cost so far: 13188.961
Iteration 3, best_cost so far: 13188.961
Iteration 4, best_cost so far: 12533.56
Iteration 5, best_cost so far: 12533.56
Iteration 6, best_cost so far: 12533.56
Iteration 7, best_cost so far: 9763.604
Iteration 8, best_cost so far: 9763.604
Iteration 9, best_cost so far: 8600.515
Iteration 10, best_cost so far: 8600.515
Iteration 11, best_cost so far: 7798.515
Iteration 12, best_cost so far: 7798.515
Iteration 13, best_cost so far: 7798.515
Iteration 14, best_cost so far: 5788.214
Iteration 15, best_cost so far: 5788.214
Iteration 16, best_cost so far: 5788.214
Iteration 17, best_cost so far: 5788.214
Iteration 18, best_cost so far: 5788.214
Iteration 19, best_cost so far: 3359.721
Iteration 20, best_cost so far: 3359.721
Iteration 21, best_cost so far: 3359.721
Iteration 22, best_cost so far: 2741.244
Iteration 23, best_cost so far: 2741.244
Iteration 24, best_cost so far: 2741.244
Iteration 25, best_cos

In [179]:
import matplotlib.pyplot as plt

# print(np.polyval(best_result, test.points[0][0]))
# print(test.points[0][1])
# Sample data for demonstration
x_values = [x[0] for x in test.points] # Replace with your actual x values
real_y_values = [x[1] for x in test.points]  # Replace with your actual real y values
list1, list2 = zip(*sorted(zip(x_values, real_y_values)))
# Convert back 
x_values = list(list1)
real_y_values = list(list2)
predicted_y_values = [np.polyval(best_result, x[0]) for x in test.points] # Replace with your actual predicted y values
predicted_y_values = sorted(predicted_y_values)
print(real_y_values)
print(predicted_y_values)
# Plot the real curve
# plt.plot(x_values, real_y_values, label='Real Curve', color='blue')

# # Plot the predicted curve
# plt.plot(x_values, predicted_y_values, label='Predicted Curve', color='red', linestyle='dashed')

# # Add labels and legend
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.title('Real vs Predicted Curve')
# plt.legend()

# # Show the plot
# plt.show()


[-379949829948.33, -293824077866.055, -188223396928.236, -185473021949.755, -177924482448.454, -136330383382.666, -100279320986.428, -52776103125.97, -49738997442.699, -46608871671.366, -45674180389.985, -26843813714.295, -22053636969.64, -20492841064.698, -17292911708.105, -12589732555.14, -8328543886.293, -69163956.292, -6957597.593, -6667993.112, -2819043.785, -1298329.036, -278255.65, -109767.257, -10454.45, -7646.216, -469.248, -0.029, 0.012, 0.116, 1271.025, 856522.613, 1138355.064, 62406775.971, 122300904.007, 270930418.949, 1432475288.482, 1439455117.674, 2518851793.41, 3126351723.071, 3180722080.763, 4620594285.391, 6746495432.931, 10292921305.449, 24513263630.634, 44237501555.532, 60662510535.694, 181076919229.972, 199723057156.746, 381827746452.189]
[-379659182685.9307, -293671782898.536, -188223855601.3563, -185422293601.90765, -177952866291.17657, -136357084278.9151, -100314553610.8545, -52847673423.58088, -49798344582.44669, -46660990868.672485, -45744004629.474915, -2688

In [104]:
# load test case
curve_test1 = Data("CurveFitting-Tests/CurveFitting_test1.txt")
curve_test1.load_data()

# create first population
population = Population(curve_test1, pop_size=100)
population.initialize_population()

# select 2 parents for crossover
father, mother = population.parent_selection(N=4)

# crossover 2 selected parents with p_cross
p_cross = 0.75
e_cross = random.random()
if e_cross < p_cross:
    child1, child2 = population.crossover(father, mother)

# mutate childs with p_mutate
p_mutate = 0.5
e_mutate1 = random.random()
if e_mutate1 < p_mutate:
    child1.mutation()
e_mutate2 = random.random()
if e_mutate2 < p_mutate:
    child2.mutation()

# population replacement
parent_values = []
child_values = []
for i in range(population.population_size):             # save all values of parents and children in 2 value lists
    x = population.population[i].fitness_function()
    y = evaluation_function(children_population[i], stock_len)
    parent_values.append(x)
    child_values.append(y)

num_replace = replacement_rate * len(population)    # find the number of parents that should be replaced with children in population

for i in range(int(num_replace)):        # replace as the size of replacements in population
    max_parent_ind = parent_values.index(max(parent_values))  # find the worst parent
    min_child_ind = child_values.index(min(child_values))      # find best children

    population.pop(max_parent_ind)      # remove worst parent from population
    population.append(children_population[min_child_ind])  # add best children to population

    children_population.pop(min_child_ind)  # remove best children from population of children
    parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
    child_values.pop(min_child_ind)  # remove best children value from children values

# find the best result in population
for i in range(len(population)):
if evaluation_function(population[i], stock_len) < best_cost:
    best_cost = evaluation_function(population[i], stock_len)
    best_result = population[i]
